In [ ]:
# © 2022. Triad National Security, LLC. All rights reserved.
# This program was produced under U.S. Government contract 89233218CNA000001 for Los Alamos
# National Laboratory (LANL), which is operated by Triad National Security, LLC for the U.S.
# Department of Energy/National Nuclear Security Administration. All rights in the program are
# reserved by Triad National Security, LLC, and the U.S. Department of Energy/National Nuclear
# Security Administration. The Government is granted for itself and others acting on its behalf a
# nonexclusive, paid-up, irrevocable worldwide license in this material to reproduce, prepare
# derivative works, distribute copies to the public, perform publicly and display publicly, and to permit
# others to do so.

In [ ]:
import os
import time
import joblib
import h5py
import random as python_random

import tensorflow as tf
import numpy as np
import scipy as sp

gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)
print(len(gpus), "Physical GPUs")

# %matplotlib widget
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams.update({'font.size': 16})
mpl.rcParams.update({'font.family': 'serif'})

from utils_data_windows import LOAD_DATA, get_dataset
from utils_data_windows import readSeismicData, get_data_scaler, data_scaler_transform, getWindowDataDict
from utils_data_windows import get_mu_slip_loc_data_with_index

from utils_labquake_model import MuModel
from utils_model_training import train_labquake_model, get_labquake_model
from utils_model_training import test_labquake_model

from sklearn.metrics import r2_score

seed_value = 1234
python_random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

In [ ]:
verbose = 1

# parameters data dict
params_data = {}
params_data['filenames_train']    = ['./Data/p4677_seismic_data_2MPa.hdf5', './Data/p4677_shear_data_2MPa.hdf5']
params_data['train_data_split']   = 0.6
params_data['valid_data_split']   = 0.4

params_data['in_scaler_mode']     = 'standard'
params_data['out_scaler_mode']    = 'minmax'
params_data['sample_freq']        = 1000.0
params_data['unit_time_intvl']    = 0.008
params_data['num_in_time_intvl']  = 1
params_data['num_out_time_intvl'] = 1
params_data['in_time_intvl']      = params_data['unit_time_intvl'] * params_data['num_in_time_intvl']
params_data['out_time_intvl']     = params_data['unit_time_intvl'] * params_data['num_out_time_intvl']
params_data['in_out_time_diff']   = 0.0
params_data['time_advance']       = params_data['out_time_intvl']

params_data['batch_size']         = 64
params_data['shuffle_buffer']     = int(1e6)

# major slip positions
p4677_train_fail_index = np.load('p4677_train_fail_index.npy')
p4677_valid_fail_index = np.load('p4677_valid_fail_index.npy')
p4581_test_fail_index = np.load('p4581_3mpa_stepup_fail_index.npy')
params_data['slip_seg_size'] = 4

In [ ]:
# Read Train/Valid data
start_time = time.time()
print('Load Data')
train_data, valid_data, input_scaler, output_scaler = LOAD_DATA(params_data)
print('\nLoad Data Time', time.time()-start_time, 's')

# Major slips
mu_slip_loc_train, mu_slip_index_train = get_mu_slip_loc_data_with_index(params_data, train_data['y_data'], p4677_train_fail_index)
print('\n')
print(mu_slip_loc_train.shape, mu_slip_index_train.shape)
print(mu_slip_index_train)

mu_slip_loc_val, mu_slip_index_val = get_mu_slip_loc_data_with_index(params_data, valid_data['y_data'], p4677_valid_fail_index)
print('\n')
print(mu_slip_loc_val.shape, mu_slip_index_val.shape)
print(mu_slip_index_val)

# Datasets
train_dataset = get_dataset(params_data, train_data['y_data'], train_data['y_data'], is_shuffle=True)
valid_dataset = get_dataset(params_data, valid_data['y_data'], valid_data['y_data'], is_shuffle=False)
options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.DATA
train_dataset = train_dataset.with_options(options)
valid_dataset = valid_dataset.with_options(options)

In [ ]:
# Read Test data
filenames_test = ['./Data/p4581_seismic_data_3mpa_stepup.hdf5', './Data/p4581_shear_data_3mpa_stepup.hdf5']

# Read data
start_time = time.time()
print('\nLoad Test Data')
signalData = readSeismicData(filenames_test)
numSignal = signalData['time'].shape[0]

# Input/output scalers
numDataScaler = int(numSignal*1.0)
input_scaler_test = get_data_scaler(signalData['ae'][:numDataScaler], params_data['in_scaler_mode'])
output_scaler_test = get_data_scaler(signalData['mu'][:numDataScaler], params_data['out_scaler_mode'])

# Scaled data
input_signal_test = data_scaler_transform(signalData['ae'], input_scaler_test)
output_signal_test = data_scaler_transform(signalData['mu'], output_scaler_test)

# Test Data
test_data = getWindowDataDict(params_data, input_signal_test, output_signal_test, signalData['time'])

print('\nLoad Data Time', time.time()-start_time, 's')

# Major slips
mu_slip_loc_test, mu_slip_index_test = get_mu_slip_loc_data_with_index(params_data, test_data['y_data'], p4581_test_fail_index)
print('\n')
print(mu_slip_loc_test.shape, mu_slip_index_test.shape)
print(mu_slip_index_test)

In [ ]:
# parameters model
params_model = {}

params_model['num_vq_codes']        = 24 + 8
params_model['embedding_dim']       = int(params_data['unit_time_intvl']*params_data['sample_freq'])
params_model['commitment_cost']     = 0.25
params_model['ema_decay']           = 0.99

params_model['save_weight_file']    = './mu_vq_latent_model.h5'
params_model['transformer_max_len'] = 1000
params_model['num_train_epochs']    = 500
params_model['learning_rate']       = 1e-3
params_model['lr_warmup_epochs']    = 10
params_model['lr_decay_epochs']     = 100
params_model['steps_per_epoch']     = int(np.ceil(train_data['y_data'].shape[0]/params_data['batch_size']))
params_model['early_stop_wait']     = 50
params_model['min_delta_stop']      = 1e-4
params_model['verbose_model_train'] = 1

In [ ]:
# 24 codes for all frictions
params_model['num_vq_codes']        = 24

# Train
mu_model_all, history = train_labquake_model(params_data, params_model, 
                                             MuModel, 
                                             train_dataset, valid_dataset)

plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], 'r-', label='train')
plt.plot(history.history['val_loss'], 'b--', label='valid')
plt.legend()

In [ ]:
# 8 codes for major slips
params_model['num_vq_codes']        = 8

# Initialize
mu_model_major_slips = get_labquake_model(params_data, params_model, 
                              MuModel, train_dataset, pre_weight=None)
print('\n')

# Manually set codes
embeddings = np.zeros((params_model['embedding_dim'], params_model['num_vq_codes']))
for ii, ind in enumerate([1, 2, 3, 6, 7, 17, 19, 29]):
  embeddings[:,ii:ii+1] = train_data['y_data'][mu_slip_index_train[ind]]
mu_model_major_slips.vq_layer.embeddings.assign(embeddings)

In [ ]:
# codebook
params_model['num_vq_codes']        = 24 + 8

mu_model = get_labquake_model(params_data, params_model, 
                              MuModel, train_dataset, pre_weight=None)

# codebook major slips
embeddings_major_slips = mu_model_major_slips.vq_layer.embeddings.numpy()

# codebook all series
embeddings_all_series = mu_model_all.vq_layer.embeddings.numpy()

# codebook assemble
embeddings = np.concatenate([embeddings_major_slips, embeddings_all_series], axis=-1)
mu_model.vq_layer.embeddings.assign(embeddings)

# save model
mu_model.save_weights(params_model['save_weight_file'])

In [ ]:
# load model
params_model['num_vq_codes']        = 24 + 8
mu_model = get_labquake_model(params_data, params_model, 
                              MuModel, train_dataset, pre_weight=params_model['save_weight_file'])

# Display codes
embeddings = mu_model.vq_layer.embeddings.numpy()
print('embeddings')
print(embeddings)
print('embeddings shape')
print(embeddings.shape)
print('\n')

fig = plt.figure(figsize=(20, 20))
for i in range(embeddings.shape[1]):
  # display original
  ax = plt.subplot(8, 8, i + 1)
  ax.plot(embeddings[:,i], 'ko-', linewidth=2)
  ax.set_xticks([])
  ax.set_ylim([-1,1])
plt.show()
print('\n')

In [ ]:
print('Training data')
y_pred, _, _ = mu_model_all(train_data['y_data'])

target_signal = np.concatenate(train_data['y_data'], axis=0)
predict_signal = np.concatenate(y_pred, axis=0)

R2_Curve = r2_score(target_signal, predict_signal)
MAPE_Curve = tf.keras.metrics.mean_absolute_percentage_error(target_signal[:,0], predict_signal[:,0])/100.0
MSE_Curve = tf.keras.metrics.mean_squared_error(target_signal[:,0], predict_signal[:,0])

print('  Full Signal R2: {:1.3}'.format(R2_Curve))
print('  Full Signal MAPE: {:1.3%}'.format(MAPE_Curve))
print('  Full Signal MSE: {:1.3}'.format(MSE_Curve))

fig = plt.figure(figsize=(30, 10))
ax = plt.gca()
ax.plot(target_signal, 'b-', linewidth=2)
ax.plot(predict_signal, 'r-', linewidth=2, alpha=1)
plt.show()
print('\n')


print('Validation data')
y_pred, _, _ = mu_model_all(valid_data['y_data'])

target_signal = np.concatenate(valid_data['y_data'], axis=0)
predict_signal = np.concatenate(y_pred, axis=0)

R2_Curve = r2_score(target_signal, predict_signal)
MAPE_Curve = tf.keras.metrics.mean_absolute_percentage_error(target_signal[:,0], predict_signal[:,0])/100.0
MSE_Curve = tf.keras.metrics.mean_squared_error(target_signal[:,0], predict_signal[:,0])

print('  Full Signal R2: {:1.3}'.format(R2_Curve))
print('  Full Signal MAPE: {:1.3%}'.format(MAPE_Curve))
print('  Full Signal MSE: {:1.3}'.format(MSE_Curve))

fig = plt.figure(figsize=(30, 10))
ax = plt.gca()
ax.plot(target_signal, 'b-', linewidth=2)
ax.plot(predict_signal, 'r-', linewidth=2, alpha=1)
plt.show()
print('\n')


print('Testing data')
y_pred, _, _ = mu_model_all(test_data['y_data'])

target_signal = np.concatenate(test_data['y_data'], axis=0)
predict_signal = np.concatenate(y_pred, axis=0)

R2_Curve = r2_score(target_signal, predict_signal)
MAPE_Curve = tf.keras.metrics.mean_absolute_percentage_error(target_signal[:,0], predict_signal[:,0])/100.0
MSE_Curve = tf.keras.metrics.mean_squared_error(target_signal[:,0], predict_signal[:,0])

print('  Full Signal R2: {:1.3}'.format(R2_Curve))
print('  Full Signal MAPE: {:1.3%}'.format(MAPE_Curve))
print('  Full Signal MSE: {:1.3}'.format(MSE_Curve))

fig = plt.figure(figsize=(30, 10))
ax = plt.gca()
ax.plot(target_signal, 'b-', linewidth=2)
ax.plot(predict_signal, 'r-', linewidth=2, alpha=1)
plt.show()
print('\n')